# CAPSNET Monitor

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
HTML(
'''<style>
.output_wrapper button.btn.btn-default,
.output_wrapper .ui-dialog-titlebar {
  display: none;
}
</style>''')

In [3]:
%matplotlib notebook

import os
#os.chdir('/home/jens/eclipse-workspace/MatrixCaps/Matrix-Capsule-Network/')

import sys
import pyrr
#sys.path.append('../model')
sys.path.append('..')
import torch
from torch.autograd import Variable
from torchvision import datasets
from torchvision import transforms
from torchvision import utils
import torch.nn.functional as F
from capsnet import CapsNet
import util
#import model.util as util
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import time
import pyrr

import ipywidgets as widgets
from ipywidgets import FloatSlider, HBox, VBox, Layout
matplotlib.pyplot.ioff()

import glob
from IPython.display import display, Markdown, Image

import PIL.Image
from io import BytesIO


## Matrix Capsules LOSS plot

In [4]:
matplotlib.rcParams.update({'font.size': 22})
fig3 = plt.figure(figsize = (9,6))
ax3 = fig3.subplots()
#matplotlib.pyplot.ion()


button = widgets.Button(description="Update")
show_epochs = widgets.IntText(description='Epoch history',disabled=False)
show_epochs.value = 30

def on_button_clicked(b):
    with open('../loss.log') as f:
        content = f.readlines()
    content = [float(x.strip()) for x in content]
    xmin = 0
    xmax = len(content)
    if xmax > show_epochs.value:
        xmin = xmax-show_epochs.value
        content = content[xmin:xmax]
    ymin = np.min(content)
    ymax = np.max(content)
    x_axis = np.linspace(xmin, xmax-1, num=xmax-xmin)

    #matplotlib.pyplot.gcf().clear()
    ax3.clear()
    ax3.set_xlim(xmin, xmax)
    yaxismin = ymin - (ymax-ymin)/3
    if yaxismin < 0:
        yaxismin = 0
    ax3.set_ylim(ymin - (ymax-ymin)/3, ymax)
    ax3.plot(x_axis, content, linewidth=5.0)
    ax3.xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))
    ax3.grid(linestyle='-', linewidth='0.5', color='red')
    fig3.canvas.draw()

    
on_button_clicked(None)
fig3.show()

display(button)
display(show_epochs)

button.on_click(on_button_clicked)
show_epochs.observe(on_button_clicked)

<IPython.core.display.Javascript object>

Button(description='Update', style=ButtonStyle())

IntText(value=30, description='Epoch history')

In [5]:
class args():
    batch_size = 1
    loss = 'MSE'
    use_recon = 0.001
    num_workers = 1
    disable_recon = False
    dataset = 'rabbit100x100'
    normalize = 0
    stereo = False

    

## Ground Truth

In [6]:
def updateOutputWidget(widget, array, color="L"):
    f = BytesIO()
    PIL.Image.fromarray(np.uint8(array*255), mode=color).convert(color).save(f, 'png')
    widget.clear_output()
    with widget:
        display(Image(data=f.getvalue(), width='100%'))


## Activations of convolutional layers and primary capsules layer

In [7]:
layer1_widget = widgets.Output(layout=Layout(width='100%'))
layer2_widget = widgets.Output(layout=Layout(width='100%'))
layer3_widget = widgets.Output(layout=Layout(width='100%'))
layer4_widget = widgets.Output(layout=Layout(width='100%'))
layer5_widget = widgets.Output(layout=Layout(width='100%'))
layer6_widget = widgets.Output(layout=Layout(width='100%'))
#layer12_widget = HBox([layer1_widget, layer2_widget], layout=Layout(width='100%'))
l345_widget = HBox([layer3_widget, layer4_widget, layer5_widget, layer6_widget])
layer_widget = VBox([layer1_widget, layer2_widget, l345_widget], layout=Layout(width='100%'))
display(layer_widget)



def makeCombinedImage(img, scale=1):
    if scale != 1:
        img = torch.nn.functional.interpolate(img, scale_factor=scale, mode='nearest')
    img = img.squeeze().unsqueeze(1)
    return utils.make_grid(img.data, nrow=int(img.shape[0] ** 0.5), normalize=True,range=(0, 1)).cpu().numpy()
    
def forward(self, x, lambda_, labels=None, draw=True):
    if x is not None:
        x = F.relu(self.bn1(self.conv1(x)))

        if draw is True:
            grid_picture = makeCombinedImage(x, scale=4)
            updateOutputWidget(layer1_widget, grid_picture.transpose(1,2,0), "RGB")

        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.bn2(self.conv2(x)))
        
        if draw is True:
            grid_picture = makeCombinedImage(x, scale=8)
            updateOutputWidget(layer2_widget, grid_picture.transpose(1,2,0), "RGB")
        
        
        # Combine left and right
        left = x[:,:,:,:int(x.shape[-1]/2)]
        right = x[:,:,:,int(x.shape[-1]/2):]
        x = torch.stack([left,right], dim=1).view(x.shape[0],-1,x.shape[2],x.shape[2])
        
        
        #x = F.relu(self.bn3(self.conv3(x)))
        #if draw is True:
        #    grid_picture = makeCombinedImage(x)
        #    updateOutputWidget(layer3_widget, grid_picture.transpose(1,2,0), "RGB")
        
        p, a = self.primary_caps(x)  # b,32*(4*4+1),12,12

        if draw is True:
            grid_picture = makeCombinedImage(a, scale=16)
            updateOutputWidget(layer3_widget, grid_picture.transpose(1,2,0), "RGB")
            
        p, a = self.convcaps1(lambda_, p, a)  # b,32*(4*4+1),5,5

        if draw is True:
            grid_picture = makeCombinedImage(a, scale=16)
            updateOutputWidget(layer4_widget, grid_picture.transpose(1,2,0), "RGB")

        p, a = self.convcaps2(lambda_, p, a)  # b,32*(4*4+1),3,3

        if draw is True:
            grid_picture = makeCombinedImage(a, scale=16)
            updateOutputWidget(layer5_widget, grid_picture.transpose(1,2,0), "RGB")

        p, a = self.convcaps3(lambda_, p, a)  # b,32*(4*4+1),3,3

        if draw is True:
            grid_picture = makeCombinedImage(a, scale=16)
            updateOutputWidget(layer6_widget, grid_picture.transpose(1,2,0), "RGB")
            
        p, a = self.classcaps(lambda_, p, a)  # b,10*16+10

        
        """ Find pose with largest activation """
        """
        dist = torch.arange(float(3)) / 3
        a_tmp = a.view(a.shape[0],-1)
        p_tmp = p.view(p.shape[0],-1,p.shape[-1])
        _, i = torch.max(a_tmp, 1)
        new_a = []
        new_p = []
        coords = []
        for counter, j in enumerate(i):
            new_a.append(a_tmp[counter, j])
            new_p.append(p_tmp[counter, j, :])

            w_x=j%3
            w_y=j/3
            xyz = p_tmp[counter, j, :][(3,7,11),]
            xyz[0] += dist[w_x]
            xyz[1] += dist[w_y]
            xyz = torch.cat([xyz, torch.FloatTensor([w_x, w_y])], dim=0)
            coords.append(xyz)
            #p[:, (3,7,11)] = xyz * 1.0

        a = torch.stack(new_a, dim=0)
        p = torch.stack(new_p, dim=0)

        coords = torch.stack(coords, dim=0)
        coords = self.coord_add_encoder(coords)
        """
        
        p = p.squeeze()
        
        # Temporary when batch size = 1
        if len(p.shape) == 1:
            p = p.unsqueeze(0)
       
        xyz = p[:, (3,7,11)]
        xyz = self.coord_add_encoder(xyz)
        p[:, (3,7,11)] = xyz * 1.0
    else:
        p = labels

    if not self.args.disable_recon:
        reconstructions = self.decoder(p)
    else:
        reconstructions = 0

    return p, reconstructions

## 4x4 pose output

In [8]:
truth_widget_left = widgets.Output(layout=Layout(width='100%'))
truth_widget_right = widgets.Output(layout=Layout(width='100%'))
recon_widget_left = widgets.Output(layout=Layout(width='100%'))
recon_widget_right = widgets.Output(layout=Layout(width='100%'))
truth_widget = HBox([truth_widget_left, truth_widget_right], layout=Layout(width='100%'))
recon_widget = HBox([recon_widget_left, recon_widget_right], layout=Layout(width='100%'))
step_widget = VBox([truth_widget, recon_widget], layout=Layout(width='100%'))
names = ['p0', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'activ']
pose_sliders = [FloatSlider(min=-100, max=100, layout={'height': '50%'}, step=0.00001, description=i, orientation='horizontal') for i in names]
button_next = widgets.Button(description="Next", layout={'height': '50%'})
pose_sliders.append(button_next)
sliders_widget = VBox(pose_sliders, layout=Layout(width='60%'))
step_slider_widget = HBox([step_widget, sliders_widget],layout=Layout(width='100%'))
pose_output = widgets.Textarea(layout={'width': '100%','height': '90px'},disabled=False)
pose_label = widgets.Textarea(layout={'width': '100%','height': '90px'},disabled=False)
pose_error = widgets.Textarea(layout={'width': '100%','height': '90px'},disabled=False)


label_widget = HBox([widgets.Label(layout={'width': '100%'},value="Output"), widgets.Label(layout={'width': '100%'},value="Label"), widgets.Label(layout={'width': '100%'},value="Error")])
pose_widget = HBox([pose_output, pose_label, pose_error])

the_time = time.time()
do_update = True

display(step_slider_widget)
display(label_widget)
display(pose_widget)

# Initialize view and projection for coord system rendering
w_width, w_height = 800, 800
focus_distance = 1.9
view = pyrr.matrix44.create_from_translation(pyrr.Vector3([0.0, 0.0, -focus_distance])).transpose()
projection = pyrr.matrix44.create_perspective_projection_matrix(65.0, w_width / w_height, 0.1, 100.0).transpose()

def plotPoint(pnt, color_idx, img):
    model_view = pyrr.matrix44.multiply(view, pnt)
    model_projection = pyrr.matrix44.multiply(projection, model_view)
    scaling = model_projection[3]
    pix_x = img.shape[1]/2 + (model_projection[0]/scaling) * img.shape[1]/2
    if pix_x >= img.shape[1]: pix_x = img.shape[1]-1
    if pix_x < 0: pix_x = 0
    pix_y = img.shape[0]/2 - (model_projection[1]/scaling) * img.shape[0]/2
    if pix_y >= img.shape[0]: pix_y = img.shape[0]-1
    if pix_y < 0: pix_y = 0
    img[int(pix_y),int(pix_x),color_idx] = 1.0
    
eye_distance = 0.5
focus_distance = 1.9
eye_angle = eye_distance/focus_distance
right_eye_transform = pyrr.Matrix44.from_y_rotation(-eye_angle/2)
left_eye_transform = pyrr.Matrix44.from_y_rotation(eye_angle/2)
    
def plotCoordSystemAxis(pose44, axis_idx, img):
    pnt = pyrr.vector4.create(0.0, 0.0, 0.0, 1.0)
    for i in range(50):
        pnt[axis_idx] = i*0.01
        point = pyrr.matrix44.multiply(pyrr.Matrix44(pose44), pnt) #.transpose()
        plotPoint(point, axis_idx, img)

pose = 0

def on_value_change(change):
    global the_time
    global do_update
    global pose
    global ground_truth_labels
    global imgs_ref
    global right_eye_transform
    global left_eye_transform

    if not do_update and change is not None:
        return

    if (time.time()-the_time) < 0.2:
        return
    the_time = time.time()
    
    for i in range(len(pose_sliders)-1):
        pose[:,i] = pose_sliders[i].value
    mat44 = util.matAffine_from_matMinRep(pose.data[:,:-1])

    reconstruction = model.image_decoder(pose.view(pose.shape[0], 1, 1, 1, -1))
    #imgs_sliced = imgs[:,[0,3],...]
    reconstruction = reconstruction.view_as(imgs_ref) #.shape[0], 1, imgs_ref.shape[2], imgs_ref.shape[3])

    recon_left = reconstruction.data.squeeze().permute(1,2,0).cpu().numpy() 
    #recon_left = reconstruction.data[:,0,:,:].squeeze().cpu().numpy()
    #recon_left = np.concatenate((recon_left[:,:,None],recon_left[:,:,None],recon_left[:,:,None]), axis=2)
    print (recon_left.shape)
    if args.stereo:
        pose_left = pyrr.Matrix44(mat44)*left_eye_transform
    else:
        pose_left = pyrr.Matrix44(mat44)
    plotCoordSystemAxis(pose_left, 0, recon_left)
    plotCoordSystemAxis(pose_left, 1, recon_left)
    plotCoordSystemAxis(pose_left, 2, recon_left)
    updateOutputWidget(recon_widget_left, recon_left, "RGB")

    if args.stereo:
        recon_right = reconstruction.data[:,1,:,:].squeeze().cpu().numpy()
        recon_right = np.concatenate((recon_right[:,:,None],recon_right[:,:,None],recon_right[:,:,None]), axis=2)
        pose_right = pyrr.Matrix44(mat44)*right_eye_transform
        plotCoordSystemAxis(pose_right, 0, recon_right)
        plotCoordSystemAxis(pose_right, 1, recon_right)
        plotCoordSystemAxis(pose_right, 2, recon_right)
        updateOutputWidget(recon_widget_right, recon_right, "RGB")

        
def showDataImage(next_data):
    global ground_truth_labels
    global imgs_ref
    global do_update
    #global ground_truth_widget
    global pose
    do_update = False
    
    """
    Prepare input
    """
    imgs_ref = next_data[0]

    ground_truth_labels = next_data[1]
    imgs_ref = Variable(imgs_ref).cpu()
    pose, reco = model(imgs_ref, disable_recon=args.disable_recon)
    pose = pose.view(pose.shape[0], -1)
    
    # Print matrices in textareas
    m_l = util.matAffine_from_matMinRep(ground_truth_labels)
    mat44 = util.matAffine_from_matMinRep(pose.data[:,:-1].cpu())
    pose_label.value = util.matrixString_from_matAffine(m_l)
    pose_output.value = util.matrixString_from_matAffine(mat44)
    pose_error.value = util.matrixString_from_matAffine(mat44 - m_l)

    # Plot coordinate systems on "truth" images
    #mat44 = util.matAffine_from_matMinRep(pose.data)
    if args.stereo:
        truth_left = imgs_ref[0,0,:,:].squeeze().numpy()
        truth_left = np.concatenate((truth_left[:,:,None],truth_left[:,:,None],truth_left[:,:,None]), axis=2)
        truth_right = imgs_ref[0,1,:,:].squeeze().numpy()
        truth_right = np.concatenate((truth_right[:,:,None],truth_right[:,:,None],truth_right[:,:,None]), axis=2)
        truth_right = imgs_ref.squeeze().permute(1,2,0) #np.concatenate((next_data[1][0,0,:,102:197,None],next_data[1][0,1,:,102:197,None],next_data[1][0,2,:,102:197,None]), axis=2)
        pose_right = pyrr.Matrix44(mat44)*right_eye_transform
        plotCoordSystemAxis(pose_right, 0, truth_right)
        plotCoordSystemAxis(pose_right, 1, truth_right)
        plotCoordSystemAxis(pose_right, 2, truth_right)
        updateOutputWidget(truth_widget_right, truth_right, "RGB")
    else:
        truth_left = imgs_ref.squeeze().permute(1,2,0) #np.concatenate((next_data[1][0,0,:,2:97,None],next_data[1][0,1,:,2:97,None],next_data[1][0,2,:,2:97,None]), axis=2)

    if args.stereo:
        pose_left = pyrr.Matrix44(mat44)*left_eye_transform
    else:
        pose_left = pyrr.Matrix44(mat44)
    plotCoordSystemAxis(pose_left, 0, truth_left)
    plotCoordSystemAxis(pose_left, 1, truth_left)
    plotCoordSystemAxis(pose_left, 2, truth_left)
    updateOutputWidget(truth_widget_left, truth_left, "RGB")

    for i in range(len(pose_sliders)-1):
        if pose[:,i] > pose_sliders[i].max:
            pose_sliders[i].max = pose[:,i]
        if pose[:,i] < pose_sliders[i].min:
            pose_sliders[i].min = pose[:,i]
            
    for i in range(len(pose_sliders)-1):
        pose_sliders[i].value = pose[:,i]
    on_value_change(None)
    do_update = True


test_dataset = util.MyImageFolder(root='../../../data/{}/test/'.format(args.dataset), transform=transforms.ToTensor(), target_transform=transforms.ToTensor(), data_rep=args.loss)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=args.batch_size, num_workers=1, shuffle=True, drop_last=False)

test_iterator = test_loader.__iter__()
imgs, labels = test_iterator.next()
test_iterator = test_loader.__iter__()

"""
Setup model, load it to CUDA and make JIT compilation
"""
imgs = imgs[:2]
model = CapsNet(dataset=args.dataset) #labels.shape[1], img_shape=imgs[0].shape, dataset=args.dataset, data_rep=args.loss, normalize=args.normalize)
model(imgs, args.disable_recon)

"""
Loading weights of previously saved states and optimizer state
"""
path="../weights/"
model_name = max(glob.iglob("{}model*.pth".format(path)),key=os.path.getctime)
display(Markdown('## Latest model is: '+model_name))
model.load_state_dict( torch.load(model_name) )
model.cpu()
model.eval()
#self = model



def on_next_button_clicked(b):
    next_data = test_iterator.next()
    showDataImage(next_data)

button_next.on_click(on_next_button_clicked)

next_data = test_iterator.next()
showDataImage(next_data)

for i in range(len(pose_sliders)):
    pose_sliders[i].observe(on_value_change)

## Latest model is: ../weights/model_1000.pth

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location='cpu' to map your storages to the CPU.